In [65]:
import json 
import csv
import datetime#import datetime module convert data to datetime64(ns)
import numpy as np
import pandas as pd#import pandas module(Better than csv module!)
import os
import glob

global train_x , train_y,train_c,TimeData,Type,Feature
train_x=[]
train_c=[]
train_y=[]
Features=np.array(['adj_close','close','high','low','open','volume'])#All the var for the model

def GenerateTrainX(n,a,c,h,l,o,v):#n for numbers /a for adj_close /o for open /h for high /l for low /c for close /v for volume
    fliter=np.array([bool(a),bool(c),bool(h),bool(l),bool(o),bool(v)])
    Choice=Features[fliter]#type(Choice)=nparray
    count=0
    for i in TimeData:
        if count<n:
            #print(i[1])
            IsWeekOrNot=int(i[1].size)/7
            if IsWeekOrNot>=5:
                train_x.append(i[1].loc[ : ,Choice].values.tolist())
                count+=1
            else:
                continue
        elif count>=n:
            break
    print("\n",train_x,"Train_x.size:  ",len(train_x))

def GenerateTrainY(n,t): #t for type of close 0 for close 1 for adj_close
    count=0
    if t==0:
        Type='close'
    elif t==1:
        Type='adj_close'
    for i in TimeData:
        if count<n:
            #print(i[1])
            IsWeekOrNot=int(i[1].size)/7
            if IsWeekOrNot>=5:
                Open=i[1].loc[:,'open'].values.tolist()[0]
                #print("Open",Open)
                Close=i[1].loc[:,Type].values.tolist()[len(i[1].loc[:,Type].values.tolist())-1]
                #print(Type,Close)
                train_y.append(float(Close-Open))
                count+=1
            else:
                continue
        elif count>=n:
            break
    print("\n",train_y,"Train_y.size:  ",len(train_y))
    
for path in glob.glob(r'./StockData/*.csv'):
    CsvData=pd.DataFrame(pd.read_csv(path))#After Reading csv put it in the dataframe
    outputData=open('./StockData/TrainingData/'+path[12:21]+'TrainingData.csv','w',newline="")
    CsvData['date']=pd.to_datetime(CsvData['date'])#將date column 原本的datatype(object) 轉成 datetime64
    CsvData["week"]=0
    for i,j in zip(CsvData["date"],range(CsvData["date"].size)):
        CsvData["week"][j]= i.isocalendar()[2]
    TimeData=CsvData.set_index('date').resample('w')
    GenerateTrainX(10,1,0,0,0,1,0)
    GenerateTrainY(10,1)
    writer=csv.writer(outputData)#寫入 csv
    writer.writerow(train_x)
    writer.writerow(train_y)
    outputData.close()
    train_x.clear()
    train_y.clear()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



 [[[20.9, 20.0], [20.9, 21.0], [20.3, 21.0], [20.4, 20.3], [20.4, 20.5]], [[20.6, 20.5], [20.0, 20.7], [19.7, 20.0], [19.8, 19.8], [19.9, 19.9]], [[20.6, 20.4], [21.3, 21.1], [20.8, 21.4], [20.9, 21.0], [20.3, 21.4]], [[19.9, 20.4], [20.4, 19.9], [20.1, 20.3], [20.1, 20.0], [20.3, 20.0]], [[20.1, 20.6], [20.0, 20.4], [19.5, 20.2], [20.2, 19.6], [20.0, 20.2]], [[0.0, 18.6], [17.3, 17.3], [17.4, 17.7], [17.2, 17.4], [17.1, 17.4]], [[18.1, 18.1], [18.3, 18.2], [18.4, 18.6], [18.1, 18.4], [18.2, 18.1]], [[18.3, 18.4], [17.9, 18.4], [17.7, 17.9], [17.8, 17.8], [17.6, 17.8]], [[18.2, 17.8], [18.6, 18.5], [18.7, 18.8], [18.2, 18.7], [18.2, 18.2]], [[18.1, 18.3], [17.8, 18.2], [17.8, 17.8], [17.7, 17.9], [17.5, 17.8]]] Train_x.size:   10

 [0.3999999999999986, -0.6000000000000014, -0.09999999999999787, -0.09999999999999787, -0.6000000000000014, -1.5, 0.09999999999999787, -0.7999999999999972, 0.3999999999999986, -0.8000000000000007] Train_y.size:   10

 [[[49.17, 49.46], [49.4, 49.21], [49.53,

## 